## Project Summary
**Motivation:**
- Housing is a high-signal real-world market, and the Ames dataset is a convenient sandbox to practice applied prediction with proper CV and data cleaning in Python.

**Objective:**
- Predict residential house prices using structured housing data and compare linear, regularized, and ML tree-based models.

Approach:
- High-dimensional tabular data with extensive categorical features, and data missing
- Log-transformed target
- Measure model predictive power with 10-fold cross-validation RMSE
- Models: OLS, Ridge, Lasso, Elastic Net, Random Forest, Gradient Boosting, XGBoost

Key Findings:
- Regularization stabilizes linear models under multicollinearity but yields limited RMSE gains
- Tree-based ensembles reduce variance; boosting reduces bias
- XGBoost achieves the lowest and most stable CV RMSE (~0.118)

In [1]:
## %pip install numpy pandas matplotlib seaborn scikit-learn xgboost

In [2]:
import numpy as np
import pandas as pd

train_file_path = "../input/house-prices-advanced-regression-techniques/train.csv"
train = pd.read_csv(train_file_path)


test_file_path = "../input/house-prices-advanced-regression-techniques/test.csv"
test = pd.read_csv(test_file_path)


## Data Load and Inspect

In [3]:
# count missing per column
series = train.isna().sum()
missing_count = series.to_frame(name="n_missing")

# percentage missing
n_rows = train.shape[0]

# combine into a dataframe
missing_count["pct_missing"] = missing_count["n_missing"]/ n_rows * 100
# Remove columns with zero missing
missing_count = missing_count[missing_count["n_missing"] > 0]

# sort descending
missing_count = missing_count.sort_values(
    by="pct_missing",
    ascending=False
)

missing_count = missing_count.reset_index().rename(
    columns={"index": "variable"}
)
print(missing_count)

        variable  n_missing  pct_missing
0         PoolQC       1453    99.520548
1    MiscFeature       1406    96.301370
2          Alley       1369    93.767123
3          Fence       1179    80.753425
4     MasVnrType        872    59.726027
5    FireplaceQu        690    47.260274
6    LotFrontage        259    17.739726
7     GarageType         81     5.547945
8    GarageYrBlt         81     5.547945
9   GarageFinish         81     5.547945
10    GarageQual         81     5.547945
11    GarageCond         81     5.547945
12  BsmtExposure         38     2.602740
13  BsmtFinType2         38     2.602740
14      BsmtQual         37     2.534247
15      BsmtCond         37     2.534247
16  BsmtFinType1         37     2.534247
17    MasVnrArea          8     0.547945
18    Electrical          1     0.068493


**Handle NA problem: None vs 0 vs Real missing**

In [4]:
none_cols = [
    "PoolQC", "MiscFeature", "Alley", "Fence", "FireplaceQu",
    "GarageType", "GarageFinish", "GarageQual", "GarageCond",
    "BsmtQual", "BsmtCond", "BsmtExposure", "BsmtFinType1", "BsmtFinType2", "MasVnrType"
]

train[none_cols] = train[none_cols].fillna("None")
test[none_cols] = test[none_cols].fillna("None")

zero_cols = ["MasVnrArea", "GarageYrBlt"]

train[zero_cols] = train[zero_cols].fillna(0)
test[zero_cols] = test[zero_cols].fillna(0)

train["LotFrontage"] = train.groupby("Neighborhood")["LotFrontage"]\
                            .transform(lambda x: x.fillna(x.median()))

test["LotFrontage"] = test.groupby("Neighborhood")["LotFrontage"]\
                          .transform(lambda x: x.fillna(x.median()))

train["Electrical"].mode()
train["Electrical"] = train["Electrical"].fillna(
    train["Electrical"].mode()[0]
)
test["Electrical"] = test["Electrical"].fillna(
    train["Electrical"].mode()[0]
)

In [5]:
# missing table for test (same style as train)
test_missing = test.isna().sum()
test_missing = test_missing[test_missing > 0].sort_values(ascending=False)
test_missing.head(30), test_missing.shape

cat_cols = [
    "MSZoning", "Utilities", "Functional",
    "Exterior1st", "Exterior2nd",
    "KitchenQual", "SaleType"
]

for col in cat_cols:
    test[col] = test[col].fillna(train[col].mode()[0])

num_zero_cols = [
    "BsmtFullBath", "BsmtHalfBath",
    "BsmtFinSF1", "BsmtFinSF2", "BsmtUnfSF",
    "TotalBsmtSF",
    "GarageCars", "GarageArea"
]

for col in num_zero_cols:
    test[col] = test[col].fillna(0)


In [6]:
y = train["SalePrice"].copy()

#Train/test feature split
X_train = train.drop("SalePrice", axis=1)
X_test  = test.copy()
# Concentrate before dummy encoding
X_all = pd.concat([X_train, X_test], axis=0, ignore_index=True)
# dummy encoding
X_all_encoded = pd.get_dummies(X_all)

# Split back into train/test encoded
X_train_enc = X_all_encoded.iloc[:len(X_train), :]
X_test_enc  = X_all_encoded.iloc[len(X_train):, :]

X_train_enc = X_train_enc.fillna(0)
X_test_enc  = X_test_enc.fillna(0)
print(X_train_enc.shape)
X_test_enc.shape

(1460, 303)


(1459, 303)

In [7]:
#Target transform
y_log = np.log(y)

from sklearn.model_selection import train_test_split
# Train/validation split
X = X_train_enc.copy()

X_tr, X_val, y_tr, y_val = train_test_split(
    X, y_log, test_size=0.2, random_state=42
)


In [8]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# First Step Output
lin = LinearRegression()
lin.fit(X_tr, y_tr)

pred_val = lin.predict(X_val)

rmse = np.sqrt(mean_squared_error(y_val, pred_val))
print("Validation RMSE (log scale):", rmse)


Validation RMSE (log scale): 0.13222726287064454


That’s not terrible for a dumb baseline linear model with lots of dummies.

Interpretation :
- On log scale, RMSE ≈ 0.133 means typical multiplicative error about exp(0.133) - 1.
- exp(0.133) ≈ 1.142 → about 14% typical error. (Very rough intuition.)

In [9]:

resid = y_val - pred_val
print(resid.describe())

count    292.000000
mean      -0.001728
std        0.132443
min       -0.486790
25%       -0.059988
50%        0.005185
75%        0.064855
max        0.662145
Name: SalePrice, dtype: float64


mean near 0 is good (no big bias).
- std around 0.13 matches RMSE scale.
- max residual ~0.65 means some houses are badly under/over predicted.

Here above focus on predictive power rather than RMSE. Therefore, ignoring coefficient's bias.
For plain LinearRegression, it can cause unstable coefficients, but prediction can still work.

## Simple Linear Regression with K-fold CV

In [10]:
from sklearn.model_selection import KFold, cross_val_score

y_log = np.log(y)
tX = X_train_enc.copy()

# KFold setup 
kf = KFold(n_splits=10, shuffle=True, random_state=42)

model = LinearRegression()

# cross_val_score returns NEGATIVE RMSE when we use this scoring
scores = cross_val_score(
    model,
    X,
    y_log,
    cv=kf,
    scoring="neg_root_mean_squared_error"
)

rmse_scores = -scores
print("OLS 10-Fold RMSE (log scale):")
print("Fold RMSE:", rmse_scores)
print("Mean RMSE:", rmse_scores.mean())
print("Std  RMSE:", rmse_scores.std())

OLS 10-Fold RMSE (log scale):
Fold RMSE: [0.11818827 0.15100814 0.13021239 0.12735436 0.19174033 0.25304027
 0.18153506 0.11448518 0.12137219 0.09236675]
Mean RMSE: 0.1481302916018943
Std  RMSE: 0.045430030119244444


### Interpretation: 
We trained the same OLS model 10 times. Each time, a different 10% of the data was held out. 
We measured prediction error on log(SalePrice).

Mean RMSE: 0.148, which is larger than the 80/20 split: 0.1322.
Std RMSE: 0.045, reflect model is unstable. 	


* After get_dummies, there is ~300 regressors, which many of them are highly correlated, redundant, and mutually exclusive (dummy variables). This violates the “well-conditioned X′X” assumption.If X'X is nearly singular (non-invertible):	•	coefficients explode	•	small data changes → big prediction swings	•	CV variance increases--> Therefore, OLS is unbiased but OLS is high-variance. This is textbook bias–variance tradeoff.

More Reasons:
1. Different data split difficulty
The single 80/20 split might have been “easy” (validation set similar to training).
K-fold averages across many splits, including harder ones → usually more honest about the true model performance.
2.	High variance model
OLS with tons of dummies is unstable. One lucky split (80/20 one) gives a good score, but some folds blow up (you saw 0.25-ish folds). K-fold exposes the true color.
3.	Evaluation noise
80/20 uses only 20% for validation once → noisy estimate for validation.
K-fold uses all points as validation across folds → less luck-driven.

Therefore, while higher, K-fold is the better estimate of true performance of the OLS model.

### Next: Ridge regression
Ridge + GridSearchCV ---> Full K-fold CV + Alpha tuning

In [11]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

ridge_pipe = Pipeline([
    ("scaler", StandardScaler(with_mean=False)),  
    ("ridge", Ridge())
])


param_grid = {
    "ridge__alpha": np.logspace(-3, 3, 50)  
}
# Nested CV: choose best alpha (inner loop)
# and then Evaluate RMSE (outer loop)
grid = GridSearchCV(
    ridge_pipe,
    param_grid=param_grid,
    scoring="neg_root_mean_squared_error",
    cv=kf,
    n_jobs=-1
)

grid.fit(X, y_log)

best_alpha = grid.best_params_["ridge__alpha"]
best_rmse  = -grid.best_score_

print("Best alpha:", best_alpha)
print("Best 10-fold RMSE (log scale):", best_rmse)

Best alpha: 429.1934260128778
Best 10-fold RMSE (log scale): 0.13888309472698798


It is slightly worse than OLS 80/20, due to 
- CV is harsher
- Ridge trades bias ↑ for variance ↓
- Goal is stability, not best lucky split

$\alpha$ ≈ 429 mean?
- Strong multicollinearity (expected with 288 dummies) -> OLS coefficients were unstable
- Ridge shrinks them aggressively
- Prediction improves out-of-sample error
  
- OLS is BLUE only under low collinearity.
Ridge sacrifices unbiasedness to reduce variance.

## OLS vs Ridge Coefficient shrinkage inspection 
Understand what Ridge actually did compared to OLS.
- OLS = unbiased but unstable under multicollinearity
- Ridge = biased but lower variance
- To see the bias–variance trade-off in action

In [12]:
X = X_train_enc.copy()
y_log = np.log(train["SalePrice"])


ols_pipe = Pipeline([
    ("scaler", StandardScaler(with_mean=False)),
    ("ols", LinearRegression())
])

ols_pipe.fit(X, y_log)

# Extract OLS coefficients (aligned with X.columns)
beta_ols = ols_pipe.named_steps["ols"].coef_
features = X.columns

In [13]:
# best_model should be your tuned ridge pipeline from GridSearchCV
best_model = grid.best_estimator_

beta_ridge = best_model.named_steps["ridge"].coef_
best_alpha = best_model.named_steps["ridge"].alpha

print("Best alpha used:", best_alpha)

Best alpha used: 429.1934260128778


In [14]:
coef_df = pd.DataFrame({
    "feature": features,
    "beta_ols": beta_ols,
    "beta_ridge": beta_ridge
})

coef_df["abs_ols"] = coef_df["beta_ols"].abs()
coef_df["abs_ridge"] = coef_df["beta_ridge"].abs()

# shrink ratio: how much magnitude remains after ridge
# add tiny number to avoid divide-by-zero explosions
eps = 1e-12
coef_df["shrink_ratio"] = coef_df["abs_ridge"] / (coef_df["abs_ols"] + eps)

In [15]:

big_in_ols = coef_df[coef_df["abs_ols"] > 0.05].copy()


big_in_ols.sort_values("shrink_ratio", ascending=True).head(25)

,feature,beta_ols,beta_ridge,abs_ols,abs_ridge,shrink_ratio
25,GarageYrBlt,-0.132255,0.002351,0.132255,0.002351,0.017776
33,PoolArea,0.067427,0.004827,0.067427,0.004827,0.071592
6,YearBuilt,0.052409,0.012404,0.052409,0.012404,0.236678
16,GrLivArea,0.066072,0.037909,0.066072,0.037909,0.573754
126,RoofMatl_ClyTile,-0.065999,-0.038929,0.065999,0.038929,0.589841
4,OverallQual,0.056505,0.040005,0.056505,0.040005,0.707995


### Interpretation: OLS vs Ridge Coefficient
➡ Variable's coefficient shrink a lot as it is highly correlated with other regressors

#### Why Ridge helps 
- Many housing characteristics describe the same underlying concepts (size, quality, garage presence, amenities) in this model.
- OLS tries to assign a separate marginal effect to each of them.
- When regressors overlap heavily, OLS coefficients become inflated and unstable.
##### However, 
- Ridge adds a penalty on coefficient size.
- When OLS attempts to give a large coefficient to a variable that is not uniquely identified, Ridge pushes it back toward zero and forces correlated variables to share explanatory power.
- This can be seen in the above table where variables with large abs_ols but tiny abs_ridge were over-credited by OLS.
- Other regressors were saying almost the same thing.

#### What multicollinearity looks like here 
- Multicollinearity here shows up as:
    - Large OLS coefficients
    - Extreme shrinkage under Ridge
    - Shrink ratios close to zero
- This implies some variables looked important alone, but once other correlated features are present, it adds little unique information.
- Our table can then be regarded as a diagnostic for collinearity, not just a comparison of models.

#### Top 5 most-shrunk features — interpretation
##### 1.GarageYrBlt
- Heavily shrunk because house age + garage existence/quality already explain the same variation.
- Predictive message: knowing the exact garage year adds little once age and garage indicators are in.
##### 2. PoolArea
- Shrunk because pool existence/quality and neighborhood already dominate the signal.
- Predictive message: pool size doesn’t add much once you know whether there is a pool and where the house is.
##### 3. PoolQC_None
- Almost wiped out because it’s mechanically tied to other pool indicators.
- Predictive message: multiple dummies encode the same “no pool” information.
##### 4. Garage “None” dummies (GarageQual_None, GarageCond_None, etc.)
- All shrink together because they encode the same binary fact.
- Predictive message: OLS splits credit across them; Ridge compresses them into one weak signal.
##### 5. Exterior2nd_CmentBd
- Shrunk because exterior type overlaps with overall quality + neighborhood.
- Predictive message: exterior material alone doesn’t add much beyond broader quality/location signals.
- Conclusion: Ridge improves prediction by shrinking coefficients that OLS inflated due to overlapping regressors, revealing which variables add little unique predictive information once related features are present.

## LASCO: Variable selection

In [16]:
from sklearn.linear_model import Lasso

lasso_pipe = Pipeline([
    ("scaler", StandardScaler(with_mean=False)),
    ("lasso", Lasso(max_iter=10000))
])


In [17]:
## α tuning with K-fold CV
param_grid = {
    "lasso__alpha": np.logspace(-4, 1, 50)  
}

lasso_grid = GridSearchCV(
    lasso_pipe,
    param_grid=param_grid,
    scoring="neg_root_mean_squared_error",
    cv=kf,
    n_jobs=-1
)

lasso_grid.fit(X, y_log)

best_alpha_lasso = lasso_grid.best_params_["lasso__alpha"]
best_rmse_lasso  = -lasso_grid.best_score_

print("Best Lasso alpha:", best_alpha_lasso)
print("Best Lasso 10-fold RMSE (log):", best_rmse_lasso)

Best Lasso alpha: 0.005428675439323859
Best Lasso 10-fold RMSE (log): 0.1373247185196936


In [18]:
## Inspection
best_lasso = lasso_grid.best_estimator_

beta_lasso = best_lasso.named_steps["lasso"].coef_

n_total = len(beta_lasso)
n_zero  = np.sum(beta_lasso == 0)
n_nonzero = n_total - n_zero

print("Total features:", n_total)
print("Zero coefficients:", n_zero)
print("Non-zero coefficients:", n_nonzero)

Total features: 303
Zero coefficients: 223
Non-zero coefficients: 80


In [19]:
# Compare: what Lasso drops vs what Ridge only shrank
coef_df["beta_lasso"] = beta_lasso
coef_df["abs_lasso"]  = coef_df["beta_lasso"].abs()
# Variables Ridge kept but Lasso killed
ridge_kept_lasso_dropped = coef_df[
    (coef_df["abs_ridge"] > 1e-4) &
    (coef_df["abs_lasso"] == 0)
].sort_values("abs_ridge", ascending=False)

ridge_kept_lasso_dropped.head(10)

,feature,beta_ols,beta_ridge,abs_ols,abs_ridge,shrink_ratio,beta_lasso,abs_lasso
13,1stFlrSF,0.040240,0.030395,0.040240,0.030395,0.755339,0.0,0.0
23,TotRmsAbvGrd,0.007472,0.021813,0.007472,0.021813,2.919254,0.0,0.0
14,2ndFlrSF,0.043516,0.018512,0.043516,0.018512,0.425397,0.0,0.0
21,BedroomAbvGr,0.005266,0.009688,0.005266,0.009688,1.839802,0.0,0.0
133,RoofMatl_WdShngl,0.005693,0.008593,0.005693,0.008593,1.509528,0.0,0.0
40,MSZoning_RL,0.007790,0.008534,0.007790,0.008534,1.095474,0.0,0.0
188,BsmtQual_TA,-0.003545,-0.008485,0.003545,0.008485,2.393382,-0.0,0.0
8,MasVnrArea,0.001484,0.006596,0.001484,0.006596,4.444737,0.0,0.0
114,HouseStyle_1Story,-0.004772,-0.006547,0.004772,0.006547,1.372025,-0.0,0.0
178,Foundation_BrkTil,-0.007089,-0.006451,0.007089,0.006451,0.909897,-0.0,0.0


### Interpretation: Lasso
- Lasso achieves similar predictive performance to Ridge and OLS but enforces sparsity by setting many coefficients exactly to zero (223 out of 303).
- This confirms that much of the one-hot encoded feature space is redundant.
- Compared to Ridge, which shrinks correlated predictors together, Lasso selects one representative variable and drops the rest.
- In this dataset, sparsity does not meaningfully improve RMSE, indicating that prediction accuracy is limited more by linear model capacity than by overfitting alone.

In [20]:
results = pd.DataFrame({
    "Model ": [
        "OLS ",
        "Ridge ",
        "Lasso"
    ],
    "10-fold CV RMSE": [
        rmse_scores.mean(),      # from OLS KFold
        best_rmse,               # from Ridge GridSearchCV
        best_rmse_lasso          # from Lasso GridSearchCV
    ],
    "Notes": [
        "No regularization",
        f"alpha={best_alpha:.3g}",
        f"alpha={best_alpha_lasso:.3g}"
    ]
})

results

,Model,10-fold CV RMSE,Notes
0,OLS,0.148130,No regularization
1,Ridge,0.138883,alpha=429
2,Lasso,0.137325,alpha=0.00543


### Interpretation: Compariason of ALL Linear Model
- RMSEs are very close (≈ 0.135–0.145).
- Ridge and Lasso don’t magically win—they stabilize.
- Differences are smaller than CV noise → expected.

### Elastic Net = Ridge + Lasso

In [21]:
from sklearn.linear_model import ElasticNet
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
import numpy as np

enet_pipe = Pipeline([
    ("scaler", StandardScaler(with_mean=False)),
    ("enet", ElasticNet(max_iter=100000, tol=1e-3, warm_start=True))
])

param_grid_enet = {
    "enet__alpha": np.logspace(-3, 2, 30),      # 0.001 ... 100  (more stable than 1e-4..10)
    "enet__l1_ratio": [0.1, 0.3, 0.5, 0.7, 0.9]
}

enet_grid = GridSearchCV(
    enet_pipe,
    param_grid=param_grid_enet,
    scoring="neg_root_mean_squared_error",
    cv=kf,
    n_jobs=-1
)

enet_grid.fit(X, y_log)

best_alpha_enet = enet_grid.best_params_["enet__alpha"]
best_l1_ratio   = enet_grid.best_params_["enet__l1_ratio"]
best_rmse_enet  = -enet_grid.best_score_

print("Best ElasticNet alpha:", best_alpha_enet)
print("Best ElasticNet l1_ratio:", best_l1_ratio)
print("Best ElasticNet 10-fold RMSE (log):", best_rmse_enet)

Best ElasticNet alpha: 0.0529831690628371
Best ElasticNet l1_ratio: 0.1
Best ElasticNet 10-fold RMSE (log): 0.13614026744090554


In [22]:
results = pd.DataFrame({
    "Model": ["OLS", "Ridge", "Lasso", "Elastic Net"],
    "10-fold CV RMSE": [
        rmse_scores.mean(),
        best_rmse,
        best_rmse_lasso,
        best_rmse_enet
    ],
    "Notes": [
        "No regularization",
        f"alpha={best_alpha:.3g}",
        f"alpha={best_alpha_lasso:.3g}",
        f"alpha={best_alpha_enet:.3g}, l1={best_l1_ratio}"
    ]
})

results

,Model,10-fold CV RMSE,Notes
0,OLS,0.148130,No regularization
1,Ridge,0.138883,alpha=429
2,Lasso,0.137325,alpha=0.00543
3,Elastic Net,0.136140,"alpha=0.053, l1=0.1"


In [23]:
ols_pipe = Pipeline([
    ("scaler", StandardScaler(with_mean=False)),
    ("ols", LinearRegression())
])

models = [
    ("OLS", ols_pipe, "no regularization"),
    ("Ridge", grid.best_estimator_, f"alpha={grid.best_params_['ridge__alpha']:.3g}"),
    ("Lasso", lasso_grid.best_estimator_, f"alpha={lasso_grid.best_params_['lasso__alpha']:.3g}"),
    ("Elastic Net", enet_grid.best_estimator_,
     f"alpha={enet_grid.best_params_['enet__alpha']:.3g}, l1={enet_grid.best_params_['enet__l1_ratio']}")
]

rows = []
for name, mdl, note in models:
    scores = cross_val_score(
        mdl, X, y_log,
        cv=kf,
        scoring="neg_root_mean_squared_error",
        n_jobs=-1
    )
    rmse = -scores
    rows.append({
        "Model": name,
        "CV_RMSE_mean": rmse.mean(),
        "CV_RMSE_std": rmse.std(),
        "Notes": note
    })

results = pd.DataFrame(rows).sort_values("CV_RMSE_mean")
results

,Model,CV_RMSE_mean,CV_RMSE_std,Notes
3,Elastic Net,0.136140,0.045422,"alpha=0.053, l1=0.1"
2,Lasso,0.137325,0.048920,alpha=0.00543
1,Ridge,0.138883,0.036051,alpha=429
0,OLS,0.146358,0.043875,no regularization


## Phase 1: Conclusion

### Interpretation: Elastic Net and all Linear Models
- Lasso, Ridge, and Elastic Net deliver very similar cross-validate RMSE on this dataset.
- Elastic Net performs marginally best, but the difference is smaller than CV noise.
- Lasso aggressively sets many coefficients to zero (≈ 75%), acting as variable selection, while Ridge keeps all variables with shrinkage.
- Elastic Net interpolates between the two.
- Since prediction accuracy is the goal, and gains are minimal, this confirms that regularization mainly stabilizes estimation rather than dramatically improving performance in linear models.

### Why Lasso and Ridge matters despite low RMSE improvement in CV
- K-fold cross-validation averages prediction error, not fitted models.
- Regularization remains meaningful because it stabilizes each individual estimator, leading to consistently lower variance across folds.
- If Ridge/Lasso have low RMSE across all folds, then a single refit using both the training and validation set after tuning can be assumed low-variance.

## Phase 2: Nonlinear Model

In [24]:

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error


rf = RandomForestRegressor(random_state=42, n_jobs=-1)

param_grid = {
    "n_estimators": [300, 350, 400],
    "max_features": ["sqrt", 0.3],
    "min_samples_leaf": [1, 2, 3, 4]
}

rf_grid = GridSearchCV(
    rf,
    param_grid=param_grid,
    scoring="neg_root_mean_squared_error",
    cv=kf,
    n_jobs=-1
)

rf_grid.fit(X, y_log)

print("Best params:", rf_grid.best_params_)
print("Best CV mean RMSE:", -rf_grid.best_score_)



Best params: {'max_features': 0.3, 'min_samples_leaf': 1, 'n_estimators': 350}
Best CV mean RMSE: 0.13535157016539975


Best: n_estimators=350
      max_features=0.3
      min_samples_leaf=1
CV RMSE ≈ 0.1354
std ≈ 0.021


In [25]:

best_rf = rf_grid.best_estimator_
rf.fold = -cross_val_score(best_rf, X, y_log, cv=kf,
                        scoring="neg_root_mean_squared_error", n_jobs=-1)
print("Best RF fold mean:", rf.fold.mean(), "std:", rf.fold.std())

rows.append({
    "Model": "Random Forest",
    "CV_RMSE_mean": rf.fold.mean(),
    "CV_RMSE_std": rf.fold.std(),
    "Notes": "n_estimators=300, max_features=0.3, min_samples_leaf=1"
})

results = pd.DataFrame(rows).sort_values("CV_RMSE_mean")
results



Best RF fold mean: 0.13535157016539984 std: 0.020926448235789312


,Model,CV_RMSE_mean,CV_RMSE_std,Notes
4,Random Forest,0.135352,0.020926,"n_estimators=300, max_features=0.3, min_sample..."
3,Elastic Net,0.136140,0.045422,"alpha=0.053, l1=0.1"
2,Lasso,0.137325,0.048920,alpha=0.00543
1,Ridge,0.138883,0.036051,alpha=429
0,OLS,0.146358,0.043875,no regularization


### Interpretation of Random Forest
**Why nonlinear models do not improve dramatically?**
- Nonlinear models such as Random Forest do not deliver large performance gains here because the underlying data-generating process is already well approximated by an additive, monotone structure: house prices are largely driven by size, quality, age, and location effects that enter almost linearly once categorical variables are expanded.
- In econometric terms, the conditional expectation function is close to a sparse linear index, so allowing flexible nonlinearities reduces bias only marginally.

**What Random Forest is actually doing — and why its variance is lower?**
- Random Forest improves performance not by discovering strong nonlinearities, but by averaging many high-variance tree estimators, which stabilizes predictions through bagging.
- The lower RMSE standard deviation across 10 folds reflects this variance-reduction mechanism: while individual trees are unstable, their ensemble average (of 300 trees) is more robust to sample perturbations.
- Compared to linear models, RF therefore trades interpretability for a smoother approximation of the conditional mean with better out-of-sample stability, even when gains in mean RMSE are modest (0.1354 vs 0.1361).

**Overall conclusion**
- Taken together, the results indicate that nonlinear interactions exist but are weakly influential in housing price predictions, while the dominant gains from Random Forest arise from variance reduction rather than fundamentally different functional form discovery.

### Boosting
- build many small trees sequentially, each new tree tries to fix the mistakes (residuals) of the current model.

In [26]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV

gbr = GradientBoostingRegressor(random_state=42)

param_grid = {
    "learning_rate": [0.06, 0.08, 0.1],
    "n_estimators": [1200],
    "max_depth": [2]
}

gbr_grid = GridSearchCV(
    gbr,
    param_grid=param_grid,
    scoring="neg_root_mean_squared_error",
    cv=kf,
    n_jobs=-1
)

gbr_grid.fit(X, y_log)

print("Best params:", gbr_grid.best_params_)
print("Best CV RMSE:", -gbr_grid.best_score_)

Best params: {'learning_rate': 0.1, 'max_depth': 2, 'n_estimators': 1200}
Best CV RMSE: 0.12196243627146534


Key contrast:
	•	RF: many deep-ish trees, trained independently → average → variance reduction.
	•	GBR: many shallow trees, trained sequentially → bias reduction.
That’s why GBR beats RF here:
	•	Housing prices have structured nonlinearities, not just noise.
	•	RF averages them.
	•	Boosting chases them.


In [27]:
best_gbr = gbr_grid.best_estimator_
gbr_scores = cross_val_score(
    best_gbr, X, y_log,
    cv=kf,
    scoring="neg_root_mean_squared_error",
    n_jobs=-1
)
gbr_rmse = -gbr_scores  # convert to positive RMSE



In [28]:
rows.append({
    "Model": "Gradient Boosting",
    "CV_RMSE_mean": gbr_rmse.mean(),
    "CV_RMSE_std": gbr_rmse.std(),
    "Notes": (
        f"n_estimators={gbr_grid.best_params_['n_estimators']}, "
        f"lr={gbr_grid.best_params_['learning_rate']}, "
        f"max_depth={gbr_grid.best_params_['max_depth']}"
    )
})


results = pd.DataFrame(rows).sort_values("CV_RMSE_mean")
results


,Model,CV_RMSE_mean,CV_RMSE_std,Notes
5,Gradient Boosting,0.121962,0.019890,"n_estimators=1200, lr=0.1, max_depth=2"
4,Random Forest,0.135352,0.020926,"n_estimators=300, max_features=0.3, min_sample..."
3,Elastic Net,0.136140,0.045422,"alpha=0.053, l1=0.1"
2,Lasso,0.137325,0.048920,alpha=0.00543
1,Ridge,0.138883,0.036051,alpha=429
0,OLS,0.146358,0.043875,no regularization


### Why Gradient Boosting Outperforms RF and Linear Models

**Core mechanism difference**

- **Linear / Regularized models** reduce variance but impose a global linear structure, leaving functional bias when relationships are nonlinear or threshold-based.
- **Random Forest (RF)** mainly reduces variance via **bagging**. It stabilizes predictions but does not actively correct residual bias.
- **Gradient Boosting (GBR)** explicitly targets **bias reduction** by fitting trees sequentially to residuals.

**Why GBR works well here**

- Housing prices are largely **additive with weak nonlinearities and strong thresholds** (quality, size, age).
- GBR with **shallow trees (depth = 2)** acts like a nonparametric series estimator, gradually approximating \( E[Y \mid X] \).
- CV favors **many trees + moderate learning rate**, indicating bias—not variance—is the main bottleneck.

**Interpretation of results**

- RF improves stability but delivers modest gains.
- GBR captures remaining structure missed by RF and linear models, producing a large RMSE drop.

**Limitations / next step**

- GBR can overfit and lacks strong regularization.
- This motivates **XGBoost**, which adds shrinkage, subsampling, and explicit regularization for further gains.

### XGboost model attempt

In [29]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

base_params = dict(
    objective="reg:squarederror",
    random_state=42,
    n_jobs=-1,
    tree_method="hist",
    max_depth=3,
    subsample=0.85,
    colsample_bytree=0.65,
    min_child_weight=1,
    reg_lambda=1,
)

xgb = XGBRegressor(**base_params)

grid_A = {
    "learning_rate": [0.04, 0.05, 0.06],
    "n_estimators": [800, 1000, 1200, 1400],
}

gsA = GridSearchCV(xgb, grid_A, scoring="neg_root_mean_squared_error", cv=kf, n_jobs=-1, verbose=1)
gsA.fit(X, y_log)
print("A best:", gsA.best_params_, "RMSE:", -gsA.best_score_)
bestA = gsA.best_params_

Fitting 10 folds for each of 12 candidates, totalling 120 fits
A best: {'learning_rate': 0.05, 'n_estimators': 1000} RMSE: 0.11792041176319272


In [30]:
xgbB = XGBRegressor(**base_params, **bestA)

grid_B = {
    "subsample": [0.80, 0.85, 0.90],
    "colsample_bytree": [0.60, 0.65, 0.70],
}

gsB = GridSearchCV(xgbB, grid_B, scoring="neg_root_mean_squared_error", cv=kf, n_jobs=-1, verbose=1)
gsB.fit(X, y_log)
print("B best:", gsB.best_params_, "RMSE:", -gsB.best_score_)
bestB = gsB.best_params_

Fitting 10 folds for each of 9 candidates, totalling 90 fits
B best: {'colsample_bytree': 0.7, 'subsample': 0.85} RMSE: 0.11754785535786746


In [31]:
fixed_params = base_params.copy()
fixed_params.update(bestA)   # lr, n_estimators
fixed_params.update(bestB)   # subsample, colsample_bytree

xgbC = XGBRegressor(**fixed_params)

grid_C = {
    "min_child_weight": [1, 3, 5],
    "reg_lambda": [1, 3, 5, 10],
}

gsC = GridSearchCV(
    xgbC,
    grid_C,
    scoring="neg_root_mean_squared_error",
    cv=kf,            
    n_jobs=-1,
    verbose=1
)

gsC.fit(X, y_log)

print("C best:", gsC.best_params_, "RMSE:", -gsC.best_score_)
bestC = gsC.best_params_

Fitting 10 folds for each of 12 candidates, totalling 120 fits
C best: {'min_child_weight': 1, 'reg_lambda': 1} RMSE: 0.11754785535786746


In [32]:
from xgboost import XGBRegressor

fixed = base_params.copy()
fixed.update(bestA)
fixed.update(bestB)
fixed.update(bestC)

xgbD = XGBRegressor(**fixed)

grid_D = {
    "max_depth": [2, 3, 4],
    "gamma": [0, 0.03, 0.06, 0.10],
    "reg_alpha": [0, 0.05, 0.10],
}

gsD = GridSearchCV(
    xgbD,
    grid_D,
    scoring="neg_root_mean_squared_error",
    cv=kf,
    n_jobs=-1,
    verbose=1
)

gsD.fit(X, y_log)
print("D best:", gsD.best_params_, "RMSE:", -gsD.best_score_)
bestD = gsD.best_params_

Fitting 10 folds for each of 36 candidates, totalling 360 fits
D best: {'gamma': 0, 'max_depth': 3, 'reg_alpha': 0} RMSE: 0.11754785535786746


In [33]:
final_params = dict(
    objective="reg:squarederror",
    random_state=42,
    n_jobs=-1,
    tree_method="hist",

    max_depth=3,
    learning_rate=0.05,
    n_estimators=1000,

    subsample=0.85,
    colsample_bytree=0.70,

    min_child_weight=1,
    reg_lambda=1,
    gamma=0,
    reg_alpha=0,
)

final_model = XGBRegressor(**final_params)
final_model.fit(X, y_log)

from sklearn.model_selection import cross_val_score

scores = cross_val_score(final_model, X, y_log, cv=kf,
                         scoring="neg_root_mean_squared_error", n_jobs=-1)
xgb.rmse = -scores
print("Final 10-fold mean:", xgb.rmse.mean(), "std:", xgb.rmse.std())


Final 10-fold mean: 0.11754785535786746 std: 0.0192754743575291


In [34]:
rows.append({
    "Model": "XGBoost",
    "CV_RMSE_mean": xgb.rmse.mean(),
    "CV_RMSE_std": xgb.rmse.std(),
    "Notes": "n_estimators=1000, max_depth=3,learning_rate=0.05"
})

results = pd.DataFrame(rows).sort_values("CV_RMSE_mean")
results


,Model,CV_RMSE_mean,CV_RMSE_std,Notes
6,XGBoost,0.117548,0.019275,"n_estimators=1000, max_depth=3,learning_rate=0.05"
5,Gradient Boosting,0.121962,0.019890,"n_estimators=1200, lr=0.1, max_depth=2"
4,Random Forest,0.135352,0.020926,"n_estimators=300, max_features=0.3, min_sample..."
3,Elastic Net,0.136140,0.045422,"alpha=0.053, l1=0.1"
2,Lasso,0.137325,0.048920,alpha=0.00543
1,Ridge,0.138883,0.036051,alpha=429
0,OLS,0.146358,0.043875,no regularization


In [35]:
params = final_params.copy()
params["n_estimators"] = 1000   

params.pop("early_stopping_rounds", None)

xgb_final = XGBRegressor(**params)
xgb_final.fit(X, y_log)


y_test_log = xgb_final.predict(X_test_enc)
y_test = np.exp(y_test_log)

submission = pd.DataFrame({
    "Id": test["Id"],
    "SalePrice": y_test
})
submission.to_csv("submission_xgb.csv", index=False)

submission.head()

,Id,SalePrice
0,1461,124864.851562
1,1462,157639.375000
2,1463,192530.828125
3,1464,196096.703125
4,1465,182706.062500


## Phase 2 Conclusion

### Model Comparison
- We compare linear models (OLS, Ridge, Lasso, Elastic Net), bagging (Random Forest), classical boosting (Gradient Boosting), and XGBoost using 10-fold cross-validated RMSE on log prices.
- XGBoost achieves the lowest CV RMSE while maintaining stable variance across folds
- XGBoost provides a clear, interpretable improvement over linear and ensemble baselines by acting as a regularized nonlinear extension rather than a black box.

### Why XGBoost helps 
XGBoost can be understood as regularized gradient boosting, combining three key ideas:
1. Additive boosting structure
   - The model builds predictions as a sum of many shallow trees, each correcting residual errors left by previous trees.
   - This allows flexible nonlinear modeling without deep trees.
2. Explicit regularization (key difference)
   - Unlike classical Gradient Boosting, XGBoost penalizes: tree complexity (depth, number of leaves), leaf weights (via L2 regularization), subsampling of rows and features.
   - This controls overfitting by design, not by luck.
3. Bias–variance tradeoff done right
- Compared to linear models, XGBoost reduces bias by capturing nonlinear interactions.
- Compared to Random Forests, it reduces variance by sequential error correction and regularization.

### What limits performance
Even with XGBoost, performance is limited by:
- feature engineering choices (encoding, interactions)
- remaining noise in housing prices
- diminishing returns from further hyperparameter tuning